# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [16]:
!pip install langchain-experimental langchain-openai faiss-cpu google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=0f930ad59f630b40d443f97c0911723b4f97b0d409fb3c525bb719a8f3069ca1
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [17]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [18]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [19]:
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('openaikey')
SERPAPI_API_KEY = userdata.get('SerpAPI')

In [20]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [21]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), prompt=todo_prompt)
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [22]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

<ipython-input-22-f5076496bc5d>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [23]:
OBJECTIVE = "Write a weather report for SF today"

In [24]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [25]:
baby_agi({"objective": OBJECTIVE})

<ipython-input-25-b5a7585b5beb>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should use the TODO action to create a todo list
Action: TODO
Action Input: "Create a todo list for writing a weather report for SF today"
Observation: 

1. Gather all necessary information: Start by collecting all the necessary information for writing a weather report for San Francisco today. This includes current temperature, humidity, wind speed, and any other relevant data.

2. Check weather forecast: Before writing the report, check the weather forecast for San Francisco today to get an idea of what to expect. This will help you accurately report on the current weather conditions.

3. Determine target audience: Consider who will be reading the weather report and tailor your writing style and language accordingly. A report for a local news station may be different from one for a scientific journal.

4. Create an outline: Organize your thoughts and i

{'objective': 'Write a weather report for SF today'}

In [39]:
# ADJUST BABYAGI TO PLAN A TRIP

# Import necessary modules
from typing import Optional
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from google.colab import userdata
import faiss
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
from serpapi import GoogleSearch
import re


# Get API keys
OPENAI_API_KEY = userdata.get('openaikey')
SERPAPI_API_KEY = userdata.get('SerpAPI')

# Define embedding model and vectorstore
embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

# Define the Chains for Trip Planning
# -----------------------------------

# 1. Trip itinerary suggestion chain
trip_prompt = PromptTemplate.from_template(
    "Plan a detailed trip itinerary for {destination}, including "
    "suggested activities, accommodation options, and transportation."
)
trip_chain = LLMChain(llm=OpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY, max_tokens=128), prompt=trip_prompt)

# 2. Search tool for gathering information
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)

# 3. Tools for the agent
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for finding information about destinations, activities, accommodations, and transportation.",
    ),
    Tool(
        name="Trip Planner",
        func=trip_chain.run,
        description="Useful for creating a detailed trip itinerary. Input: destination. Output: a detailed itinerary.",
    ),
]

# Agent setup
prefix = """You are a helpful AI travel agent assisting with trip planning.
Your objective is: {objective}.
Consider these completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

llm = OpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)


# Run the BabyAGI for Trip Planning
# ---------------------------------

OBJECTIVE = "Plan a trip to Tokyo, Japan."  # New objective

# Extract destination and duration using regular expressions
match = re.search(r"Plan a (\d+)-day trip to (.*)", OBJECTIVE)
if match:
    duration = int(match.group(1))
else:
    # Handle case where destination and duration are not found in OBJECTIVE
    print("Could not extract destination and duration from objective.")
    destination = "Unknown"

verbose = False
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

# Pass destination and duration to baby_agi
baby_agi({"objective": OBJECTIVE, "destination": destination})

Could not extract destination and duration from objective.

*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: It's important to prioritize tasks and create a list to stay organized.
Action: Trip Planner
Action Input: Tokyo, Japan
Observation: 

Day 1:

- Arrive at Tokyo's Narita International Airport in the morning
- Take the Narita Express train to Tokyo Station
- Check into your accommodation at Hotel Metropolitan Tokyo Marunouchi
- Have lunch at a nearby ramen shop
- Explore the Ginza district, known for its high-end shopping and dining
- Visit the Tsukiji Fish Market, one of the world's largest fish markets
- Dinner at a traditional izakaya (Japanese pub)

Day 2:

- Start the day at Asakusa, a historic district with temples and street vendors
- Visit Senso-ji
Thought: It's important to learn about the culture and history of the destination.
Action: Search
Action Input: Asakusa, Japan
Observation: ['As

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4141 tokens (3885 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}